<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [102]:
import pandas as pd
import psycopg2

In [103]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [104]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [40]:
# текст запроса
query_3_1 = f'''
select count(v.id) vacancies_count
from vacancies v
'''

In [42]:
# результат запроса
vacancies = pd.read_sql_query(query_3_1, connection)
display(vacancies)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\266045152.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [43]:
# текст запроса
query_3_2 = f'''
select count(e.id) employers_count
from employers e
'''

In [44]:
# результат запроса
employers = pd.read_sql_query(query_3_2, connection)
display(employers)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1330737098.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [45]:
# текст запроса
query_3_3 = f'''
select count(a.id) areas_count
from areas a
'''

In [46]:
# результат запроса
areas = pd.read_sql_query(query_3_3, connection)
display(areas)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1120855833.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [47]:
# текст запроса
query_3_4 = f'''
select count(i.id)
from industries i
'''

In [48]:
# результат запроса
industries = pd.read_sql_query(query_3_4, connection)
display(industries)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\4064102564.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,294


***

## Выводы по предварительному анализу данных
Перед нами представлена достаточно обширная БД вакансий, состоящая из 49197 вакансий от 23501 работодателя, представленных в 1362 регионах, включающих в себя 294 отрасли. 
Можно отметить, что БД включает в себя вакансии не только из России, представлены и другие страны, однако, как правило, без указания города.
Более информативные выводы можно сделать на основе детального анализа вакансий и группировки данных по разным критериям.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [49]:
# текст запроса
query_4_1 = f'''
select a.name area, count(v.name) cnt
from areas a 
join vacancies v on a.id = v.area_id
group by area
order by cnt desc
'''

In [50]:
# результат запроса
vacancies_by_cities = pd.read_sql_query(query_4_1, connection)
display(vacancies_by_cities)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\641841304.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [51]:
# текст запроса
query_4_2 = f'''
select count(v.id) vacancies_with_salary
from vacancies v
where (salary_to is not null) or (salary_from is not null)
'''

In [52]:
# результат запроса
not_null_salary = pd.read_sql_query(query_4_2, connection)
display(not_null_salary)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\3021439578.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,vacancies_with_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [53]:
# текст запроса
query_4_3 = f'''
select round(avg(salary_from),0) avg_salary_from, round(avg(salary_to),0) avg_salary_to
from vacancies
where (salary_to is not null) or (salary_from is not null)
'''

In [54]:
# результат запроса
avg_low_high_salary = pd.read_sql_query(query_4_3, connection)
display(avg_low_high_salary)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\2563983243.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [55]:
# текст запроса
query_4_4 = f'''select schedule, employment, count(id) vacancies_count
from vacancies
group by schedule, employment
order by 3 desc
''' 

In [56]:
# результат запроса
schedule_with_employment = pd.read_sql_query(query_4_4, connection)
display(schedule_with_employment)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\4024654974.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [57]:
# текст запроса
query_4_5 = f'''select experience, count(id) vacancies_count
from vacancies
group by experience
order by 2
'''

In [58]:
# результат запроса
experience = pd.read_sql_query(query_4_5, connection)
display(experience)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\312673723.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы по детальному анализу вакансий 
1. Количество представленных вакансий в разрезе городов прямо пропорционально населению города: основное количество сконцентрировано
в городах федерального значения (Москва, Санкт-Петербург), столицах соседних государств (Минск, Ташкент, Нур-Султан) и городах-миллионниках
России.
2. Средний диапазон указываемой зарплаты находится в интервале от 70 до 110 тысяч рублей. По данным Росстата, средняя зарплата по России составляла 73709 руб. в 2023 году и 86951 руб. в 2024 году. Можно предположить, что по вакансиям
из данных выше среднестатистической, однако, стоит учитывать, что хотя бы одна граница зарплаты указана всего лишь примерно в половине вакансий;
данные по вакансиям с неуказанной зарплатой, скорее всего, скорректируют данные в сторону понижения, т.к. работодатель больше склонен указывать
зарплату при ее значении выше среднего. Также стоит учитывать наличие вакансий вне России, где значения могут быть достаточно разнящимися,
требующими дополнительного анализа (возможны выбросы при построении модели), которые могут сдвинуть границы вверх от среднестатистических показателей.

3. Однозначным предпочтением для работодателя является кандидат, устраивающийся на полный рабочий день с полной занятостью (примерно 70 % вакансий). Если рассматривать тип рабочего графика и тип трудоустройства по отдельности, то в рабочем графике требование к полному дню остается примерно в том же процентном диапазоне; требование полной занятости возрастает еще сильнее и составляет более 90% (!). Также можно отметить удаленный график работы, занимающий приблизительно 20% от общего числа вакансий и, скорее всего, со временем, продолжающий расти.

4. Основной требуемый опыт в вакансиях составляет от 1 до 6 лет (приблизительно 80% от общего числа вакансий), что характеризует начальный и средний опыт работы, превалирующий среди требований работодателей. Около 15% представленных вакансий не требуют начального опыта. Меньше всего результатов по опыту работу от 6 лет; причина может крыться в небольшом наличии таковых вакансий в целом среди возможных, т.к. большой опыт работы не подразумевает под собой линейную должность, также закрытие таких вакансий может осуществляться работодателями через знакомых по 
рекомендациям.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [59]:
# текст запроса
query_5_1 = f'''(select 1 place, e.name company_name, count(v.id) vacancies_count
from vacancies v
join employers e on v.employer_id = e.id
group by e.name
order by 3 desc
limit 1)
union
(select 5, e.name, count(v.id)
from vacancies v
join employers e on v.employer_id = e.id
group by e.name
order by 3 desc
limit 1 offset 4)
order by 2 desc
'''

In [60]:
# результат запроса
top_employers = pd.read_sql_query(query_5_1, connection)
display(top_employers)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1908966189.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,place,company_name,vacancies_count
0,1,Яндекс,1933
1,5,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [61]:
# текст запроса
query_5_2 = f'''select a.name area, count(v.id) vacancies_count, count(e.id) employers_count
from areas a
left join vacancies v on a.id = v.area_id 
left join employers e on a.id = e.area
group by a.name
order by 2, 3 desc
limit 1
''' 

In [62]:
# результат запроса
employers_vacancies = pd.read_sql_query(query_5_2, connection)
display(employers_vacancies)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1157650841.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,vacancies_count,employers_count
0,Россия,0,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [63]:
# текст запроса
query_5_3 = f'''select 
    e.name company_name, count(distinct v.area_id) areas_count
from employers e 
join vacancies v on e.id = v.employer_id
group by e.id
order by 2 desc
'''

In [64]:
# результат запроса
employers_areas = pd.read_sql_query(query_5_3, connection)
display(employers_areas)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\2458500065.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,company_name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [65]:
# текст запроса
query_5_4 = f'''select count(*) employers_without_sphere
from employers e
left join employers_industries ei on e.id = ei.employer_id
where ei.industry_id is null
'''

In [66]:
# результат запроса
without_sphere = pd.read_sql_query(query_5_4, connection)
display(without_sphere)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\3522599445.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers_without_sphere
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [67]:
# текст запроса
query_5_5 = f'''select
    3 place, e.name company_name, count(ei.industry_id) sphere_count
from employers e 
join employers_industries ei on e.id = ei.employer_id
group by e.id
having count(ei.industry_id)=4
order by 2
limit 1 offset 2
'''

In [68]:
# результат запроса
four_spheres = pd.read_sql_query(query_5_5, connection)
display(four_spheres)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\3457229074.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,place,company_name,sphere_count
0,3,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [69]:
# текст запроса
query_5_6 = f'''
select count(distinct ei.employer_id) employers_count
from employers_industries ei
join industries i on ei.industry_id = i.id
where name like 'Разработка программного%'
'''

In [70]:
# результат запроса
software_sphere = pd.read_sql_query(query_5_6, connection)
display(software_sphere)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\2715930654.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,employers_count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [71]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
tabl = list(page.find("table", class_="standard").find_all('a'))[5:]
cities = []
for i in range(len(tabl)):
    city = str(tabl[i]).split('"')[-2]
    cities.append(city)
cities = tuple(cities)
print(cities)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [75]:
# текст запроса
query_5_7 = f'''
select a.name area, count(v.id) vacancies_cnt 
from employers e 
join vacancies v on v.employer_id = e.id
join areas a on v.area_id = a.id
where e.name = 'Яндекс'
group by a.name
having a.name in {cities}
union 
select 
'Total', count(v.id)
from employers e 
join vacancies v on v.employer_id = e.id
join areas a on v.area_id = a.id
where e.name = 'Яндекс' and a.name in {cities}
order by 2
'''

In [76]:
# результат запроса
yandex_mill_cities = pd.read_sql_query(query_5_7, connection)
display(yandex_mill_cities)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\364762015.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,area,vacancies_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

# Выводы по анализу работодателей
Наибольшее количество вакансий представлено от работодателей-гигантов, таких как Яндекс, Газпром, 2ГИС и другие. Они же представлены в наибольшем количестве регионов.
Работодатели с одним регионом, как правило, являются локальными, в то время как "гиганты" дают возможность к переезду в другие регионы.

Много работодателей без указания сферы деятельности, что затрудняет качественный анализ в данном сегменте. В тот же момент это может мешать найти им хорошего сотрудника среди достаточно большой конкуренции.
Работодатели с большим количеством сфер деятельности наоборот расширяют свой диапазон поиска сотрудников, указывая при этом кто конкретно им нужен, что помогает качественно работать по кандидатам у вакансии. 

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [79]:
# текст запроса
query_6_1 = f'''select count(v.id) data_vacancies_cnt
from vacancies v
where lower(v.name) like '%data%' or lower(v.name) like '%данн%'
'''

In [80]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
display(data_vacancies)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1964759077.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,data_vacancies_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [85]:
# текст запроса
query_6_2 = f'''select count(v.id) junior_ds_vacancies_cnt
from vacancies v
where 
(lower(v.name) like '%data scientist%'
or lower(v.name) like '%data science%'
or lower(v.name) like '%исследователь данных%'
or (lower(v.name) like '%ml%' and lower(v.name) not like '%html%')
or lower(v.name) like '%machine learning%'
or lower(v.name) like '%машинн%обучен%')
and
(lower(v.name) like '%junior%' or
v.experience = 'Нет опыта' or 
v.schedule = 'Стажировка')
'''

In [87]:
# результат запроса
vacancies_for_data_junior = pd.read_sql_query(query_6_2, connection)
display(vacancies_for_data_junior)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\2094735996.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,junior_ds_vacancies_cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [88]:
# текст запроса
query_6_3 = f'''select count(v.id) sql_skill_cnt
from vacancies v
where 
(lower(v.name) like '%data scientist%'
or lower(v.name) like '%data science%'
or lower(v.name) like '%исследователь данных%'
or (lower(v.name) like '%ml%' and lower(v.name) not like '%html%')
or lower(v.name) like '%machine learning%'
or lower(v.name) like '%машинн%обучен%')
and 
(lower(v.key_skills) like '%sql%' or 
lower(v.key_skills) like '%postgres%')
'''

In [89]:
# результат запроса
sql_skill = pd.read_sql_query(query_6_3, connection)
display(sql_skill)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1992121662.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,sql_skill_cnt
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [90]:
# текст запроса
query_6_4 = f'''select count(v.id) python_skill_cnt
from vacancies v
where 
(lower(v.name) like '%data scientist%'
or lower(v.name) like '%data science%'
or lower(v.name) like '%исследователь данных%'
or (lower(v.name) like '%ml%' and lower(v.name) not like '%html%')
or lower(v.name) like '%machine learning%'
or lower(v.name) like '%машинн%обучен%')
and 
(lower(v.key_skills) like '%python%')
'''

In [91]:
# результат запроса
python_skill = pd.read_sql_query(query_6_4, connection)
display(python_skill)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1080388081.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,python_skill_cnt
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [92]:
# текст запроса
query_6_5 = f'''select round(avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1),2) avg_skills_cnt
from vacancies v
where 
(
       (v.name ilike '%data scientist%')
    OR (lower(v.name) ilike '%data science%')
    OR (v.name ilike '%исследователь данных%')
    OR (v.name  like '%ML%' and  (v.name not ilike '%html%'))
    OR (v.name ilike '%machine learning%')
    OR (v.name ilike '%машинн%обучен%')
    AND (key_skills is not null)
)
'''

In [93]:
# результат запроса
average_skills = pd.read_sql_query(query_6_5, connection)
display(average_skills)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\2703521546.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,avg_skills_cnt
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [100]:
# текст запроса
query_6_6 = f'''select
v.experience, round(avg((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from))/2)) avg_salary
from vacancies v
WHERE
(v.salary_from is not null or v.salary_to is not null) and 
(v.name ilike '%data science%' or 
v.name ilike '%data scientist%' or
(v.name like '%ML%' and v.name not ilike '%html%') or 
v.name ilike '%machine learning%' or 
v.name ilike '%машинн%обучен%')
group by v.experience
'''

In [101]:
# результат запроса
salary_for_exper = pd.read_sql_query(query_6_6, connection)
display(salary_for_exper)

C:\Users\HP\AppData\Local\Temp\ipykernel_7792\1940949666.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

# Выводы по предметному анализу
В целом, количество вакансий для дата-сайентиста (536) не очень велико среди всех и составляет немногим более 1% от общего количества. Это может быть обосновано ненужностью вакансии в отдельных компаниях (небольших, не работающих с большими данными) и средним процентом занятых по отраслям, для чего нужен отдельный анализ нехватки как рабочих мест, так и кандидатов на предлагаемые должности.

Среди обязательных навыков популярен как Python (более 70% вакансий), так и SQL (порядка 50% от общего числа). В целом среднее количество навыков для должности равно 6,41; среди этих навыков могут присутствовать как технические, 2 из которых уже описаны выше, так и "гибкие" для поддержания устойчивого "морального духа" компании, как и в любой отрасли.

Среди всех вакансий дата-сайентиста для начинающего специалиста предлагается 51 вакансия, что составляет 10% от общего числа. Конкуренция входа на рынок достаточно высока, однако наличие свободных вакансий показывает расширение в области DS.  

Вместе с этим следует отметить более высокую стоимость оплаты труда в сегменте, чем по рынку. Для специалистов без опыта предлагается оплата труда чуть выше средней нижней зарплатной вилки, однако с опытом работы в сфере от года уровень дохода предлагается уже выше, чем средний рыночный в целом.
Для специалистов с опытом от 3 лет средняя оплата труда составляет около 250 тыс.руб., что можно охарактеризовать как высокий уровень дохода.

# Общий вывод по проекту

Проведен общий анализ по вакансиям, размещенным на сайте hh.ru.
Подсчитаны общие показатели выборки, после чего проведены более детальные исследования с уклоном на выборку для дата-сайентистов с помощью запросов SQL и приведения их к DataFrame в Pandas.

Хотелось бы отметить, что для заключения точных данных выборка недостаточна (всего 50000 вакансий), однако, дает общее представление о зарплатах, распределении вакансий и работодателях. Для более точного анализа с построением дальнейшей модели можно в будущем:
* увеличить выборку, добавив больше вакансий
* соединить данные с таблицами по резюме, размещенными на аналогичном сайте для более точного предсказания движения рынка в сторону дефицита/избытка вакансий, роста/снижения зарплат по отдельным отраслям

Также хочется отметить недостаточность некоторых данных, таких как "сферы деятельности" у работодателя и указание зарплатной вилки вакансии. Часть данных можно попытаться привести в более точный вид путем "ручного перебора" отдельных записей с пустыми значениями, однако, для качественного построения модели немало данных придется удалить из результирующей выборки, чтобы впоследствии не получить искаженную модель для какого-либо исследования. 

Среди уже присутствующих БД можно дополнительно провести исследования в области:
* зарплат для других сфер деятельности
* попробовать выявить зависимости роста/снижения предлагаемых зарплат в зависимости от региона
* найти дополнительные требования к сотрудникам в области "скилов" в зависимости от сферы деятельности, чтобы впоследствии рекомендовать кандидатам на отдельные должности обратить внимание на "пробелы" или, напротив, "сильные стороны" как преимущество
* разделить работодателей на "гигантов" и остальных, чтобы впоследствии выявить зависимость в зарплатах среди 2 признаков
